In [ ]:
import os
import os.path as op
import numpy as np
import pandas as pd

import nibabel as nib
from nilearn import datasets
from nimare.transforms import p_to_z

import nilearn
from nilearn import plotting
import nilearn.reporting
from nilearn.image import threshold_img

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data_dir = "./dset"
abide_dir = op.join(data_dir, "group/habenula")
clust_dir = op.join(abide_dir, "clustsim")

In [ ]:
sns.set_theme(color_codes=True)

#### Use Nilearn to identify peak clusters from z thresholded group difference correlation maps that will be used for masking

In [ ]:
# get peaks
brik_fn = op.join(
    abide_dir, "sub-group_task-rest_desc-1S2StTesthabenula_briks+tlrc.BRIK"
)
table_fn = op.join(abide_dir, "sub-group_task-rest_desc-1S2StTesthabenula_table.txt")
nii_1s_fn = op.join(abide_dir, "sub-group_task-rest_desc-2SampletTest_zmap.nii.gz")
cluster_fn = op.join(clust_dir, "clustsim_out.NN3_bisided.1D")

column_names = [".10000", ".05000", ".02000", ".01000"]
cluster_df = pd.read_table(
    cluster_fn, skiprows=8, delim_whitespace=True, names=column_names
)
cluster_df = cluster_df.reset_index()
cluster_df.rename(columns={"index": "pthr"}, inplace=True)
print(cluster_df)

cmap = "cool"
brik_idx = [11]
nii_fns = [nii_1s_fn]
tests = ["1s"]
alpha = ".02000"
pthrs = [0.0001]
cohen_thresh = 0

data_df = pd.read_csv(table_fn, sep="\t")
n_sub = data_df.groupby("group").size().sum()
n_sub_1, n_sub_2 = data_df.groupby("group").size().values

for brik_i, nii_fn, test, pthr in zip(brik_idx, nii_fns, tests, pthrs):
    convert = f"3dAFNItoNIFTI \
        -prefix {nii_fn} \
        {brik_fn}[{brik_i}]"
    os.system(convert)

    nii_img = nib.load(nii_fn)
    z_thresh = p_to_z(pthr)
    info = nii_img.get_fdata()

    clust_ext = cluster_df.loc[cluster_df["pthr"] == pthr, alpha].values[0]
    nii_thr_img = threshold_img(nii_img, z_thresh, cluster_threshold=clust_ext)
    print(clust_ext, pthr, z_thresh)

    clusters = nilearn.reporting.get_clusters_table(nii_thr_img, z_thresh, two_sided=True)
    print(clusters)  # coordinates are same as affine of input (MNI)

### Atlases
#### load Harvard and Juliech Atlases 

In [ ]:
from nilearn import datasets
from nilearn.image import load_img
from nilearn.input_data import NiftiLabelsMasker
import numpy as np

In [ ]:
# Load the Juelich atlas
juelich_atlas = datasets.fetch_atlas_juelich("maxprob-thr0-1mm")
atlas_filename = juelich_atlas.maps
atlas_labels = juelich_atlas.labels

# Load the Harvard-Oxford atlas
harvard_oxford_atlas = datasets.fetch_atlas_harvard_oxford("cort-maxprob-thr25-1mm")
atlas_filename = harvard_oxford_atlas.maps
atlas_labels = harvard_oxford_atlas.labels

# Load the atlas image
atlas_img = load_img(atlas_filename)

In [ ]:
# Function to find the region for a given coordinate
def find_region(coord):
    # Transform MNI coordinates to voxel indices
    voxel_indices = np.round(
        nib.affines.apply_affine(np.linalg.inv(atlas_img.affine), coord)
    ).astype(int)
    # Get the label index for the voxel
    label_idx = atlas_img.get_fdata()[tuple(voxel_indices)]
    region_name = (
        atlas_labels[int(label_idx)]
        if int(label_idx) < len(atlas_labels)
        else "Unknown"
    )
    return region_name


# Extract the coordinates from the clusters table and map to regions
def map_clusters_to_regions(clusters_df):
    coordinates = clusters_df[["X", "Y", "Z"]].values
    regions = [find_region(coord) for coord in coordinates]
    clusters_df["Region"] = regions
    return clusters_df

In [ ]:

# Iterate over your clusters and map to regions
for brik_i, nii_fn, test, pthr in zip(brik_idx, nii_fns, tests, pthrs):
    convert = f"3dAFNItoNIFTI -prefix {nii_fn} {brik_fn}[{brik_i}]"
    os.system(convert)

    nii_img = nib.load(nii_fn)
    z_thresh = p_to_z(pthr)
    info = nii_img.get_fdata()

    clust_ext = cluster_df.loc[cluster_df["pthr"] == pthr, alpha].values[0]
    nii_thr_img = threshold_img(nii_img, z_thresh, cluster_threshold=clust_ext)
    print(clust_ext, pthr, z_thresh)

    clusters = nilearn.reporting.get_clusters_table(
        nii_thr_img, stat_threshold=z_thresh, two_sided=True
    )
    clusters_with_regions = map_clusters_to_regions(clusters)
    print(clusters_with_regions)

    output_filename = op.join(
        abide_dir, f"clusters_with_regions_{test}_pthr_{pthr}.csv"
    )
    clusters_with_regions.to_csv(output_filename, index=False)
    print(f"Saved clusters with regions to {output_filename}")

### Create dataframes for Statistical Significance

In [ ]:
# Load phenotype correlation data
corr_df = pd.read_csv(op.join(abide_dir, "pheno4clstr-correlation2.csv"))
corr_df = corr_df[corr_df["score"] != -9999.0]  # Remove rows with score == -9999.0


# Calculate the mean Correlation for each Subject, cluster, and phenotype
mean_corr_df = corr_df.groupby(
    ["Subject", "Group", "Age", "Cluster", "phenotype", "score"], as_index=False
)["Correlation"].mean()

# Pivot the DataFrame to have phenotypes as columns with the mean score values
pivot_df = mean_corr_df.pivot(
    index=["Subject", "Group", "Age", "Cluster", "Correlation"],
    columns="phenotype",
    values="score",
)

# Flatten the columns and reset the index to get a clean DataFrame
pivot_df.columns.name = None
pivot_df.reset_index(inplace=True)

pivot_df.rename(
    columns={
        "Correlation": "RSFC",
        "ADOS_GOTHAM_SOCAFFECT": "Phen1",
        "ADOS_GOTHAM_RRB": "Phen2",
        "SRS_MOTIVATION": "Phen3",
        "VINELAND_DAILYLVNG_STANDARD": "Phen4",
        "VINELAND_COPING_V_SCALED": "Phen5",
        "VINELAND_RECEPTIVE_V_SCALED": "Phen6",
        "VINELAND_EXPRESSIVE_V_SCALED": "Phen7",
        "VINELAND_WRITTEN_V_SCALED": "Phen8",
        "VINELAND_COMMUNICATION_STANDARD": "Phen9",
        "SRS_COMMUNICATION": "Phen10",
    },
    inplace=True,
)
# Load participant.tsv
participants_df = pd.read_csv(
    op.join(data_dir, "participants.tsv"),
    sep="\t",
    usecols=["participant_id", "SITE_ID", "SEX"],
)
# Rename the columns
participants_df.rename(
    columns={"participant_id": "Subject", "SITE_ID": "Site", "SEX": "Sex"}, inplace=True
)

# Merge the two DataFrames on Subject
merged_df = pd.merge(pivot_df, participants_df, on="Subject")

# Reorder the columns
ordered_columns = [
    "Subject",
    "Cluster",
    "Group",
    "Age",
    "Sex",
    "Site",
    "RSFC",
    "Phen1",
    "Phen2",
    "Phen3",
    "Phen4",
    "Phen5",
    "Phen6",
    "Phen7",
    "Phen8",
    "Phen9",
    "Phen10",
]

pheno_df = merged_df[ordered_columns]

# Display the reordered DataFrame
print(pheno_df)
clusters = [1, 2, 3, 4]

for cluster in clusters:
    cluster_df = pheno_df[
        pheno_df["Cluster"] == cluster
    ].copy()  # Filter for current cluster

    # Drop the cluster column
    cluster_df.drop(columns=["Cluster"], inplace=True)

    # Drop rows with NaN values
    # cluster_df.dropna(axis=0, inplace=True)

    # Save the filtered DataFrame to a CSV file
    output_path = op.join(abide_dir, f"cluster-{cluster}_data.csv")
    cluster_df.to_csv(output_path, index=False)

    # Print the filtered DataFrame for verification
    print(cluster_df)

### Plotting Correlations as a function of Phenotypic Score

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

In [ ]:
# Define the r2 function to calculate R^2
def r2(x, y):
    return stats.pearsonr(x, y)[0] ** 2

In [ ]:
# rename columns of pheno df to what you want to be in your plots

pheno_df.rename(
    columns={
        "RSFC": "Correlation (Fisher's Z)",
        "Phen1": "ADOS_GOTHAM_SOCAFFECT",
        "Phen2": "Restricted and Repetitive Behaviors Total Subscore for Gotham Algorithm of the ADOS",
        "Phen3": "Social Responsivenes Scale Social Motivation Subscore Raw Total",
        "Phen4": "Vineland Adaptive Behavior Scales Daily Living Skills Standard Score",
        "Phen5": "VINELAND_COPING_V_SCALED",
        "Phen6": "Vineland Adaptive Behavior Scales Receptive Language V Scaled Score",
        "Phen7": "VINELAND_EXPRESSIVE_V_SCALED",
        "Phen8": "VINELAND_WRITTEN_V_SCALED",
        "Phen9": "VINELAND_COMMUNICATION_STANDARD",
        "Phen10": "SRS_COMMUNICATION",
    },
    inplace=True,
)

print(pheno_df)

# Melt the DataFrame to long format
plotcorr_df = pd.melt(
    pheno_df,
    id_vars=[
        "Subject",
        "Cluster",
        "Group",
        "Age",
        "Sex",
        "Site",
        "Correlation (Fisher's Z)",
    ],  # Assuming 'Subject' is the identifier column
    value_vars=[
        "ADOS_GOTHAM_SOCAFFECT",
        "Restricted and Repetitive Behaviors Total Subscore for Gotham Algorithm of the ADOS",
        "Social Responsivenes Scale Social Motivation Subscore Raw Total",
        "Vineland Adaptive Behavior Scales Daily Living Skills Standard Score",
        "VINELAND_COPING_V_SCALED",
        "Vineland Adaptive Behavior Scales Receptive Language V Scaled Score",
        "VINELAND_EXPRESSIVE_V_SCALED",
        "VINELAND_WRITTEN_V_SCALED",
        "VINELAND_COMMUNICATION_STANDARD",
        "SRS_COMMUNICATION",
    ],
    var_name="Phenotype",
    value_name="Score",
)

print(plotcorr_df)

In [ ]:
def plot_cluster_correlation(
    phenotype_df, phenotype, cluster, region, custom_palette, custom_palette2
):
    # Calculate x-axis limits based on score column
    xlim_min = phenotype_df["Score"].min() - 1  # Minus 1 from the lowest score
    xlim_max = phenotype_df["Score"].max() + 1  # Plus 1 to the highest score

    # Create a joint plot
    g = sns.jointplot(
        data=phenotype_df,
        x="Score",
        y="Correlation (Fisher's Z)",
        hue="Group",
        palette=custom_palette,
        kind="scatter",
        height=7,
    )

    # Adding regression lines for each group
    for i, (group_name, group_data) in enumerate(phenotype_df.groupby("Group")):
        sns.regplot(
            x="Score",
            y="Correlation (Fisher's Z)",
            data=group_data,
            scatter=False,
            ax=g.ax_joint,
            color=custom_palette2[group_name],  # Match color with the group
        )

        # Calculate R^2 for the current group's regression line
        r_squared = r2(group_data["Score"], group_data["Correlation (Fisher's Z)"])

        # Annotate the plot with the R^2 value, with box around it
        # Position annotations vertically
        y_offset = i * 0.06  # Adjust vertical spacing between annotations
        g.ax_joint.annotate(
            f"$R^2$ = {r_squared:.4f}",
            xy=(0.1, 0.9 - y_offset),  # Adjust the position of the annotation (x, y)
            xycoords="axes fraction",
            ha="left",
            va="top",  # Horizontal and vertical alignment
            fontsize=10,
            color=custom_palette2[group_name],  # Match color with the group
            bbox=dict(
                boxstyle="round,pad=0.3",
                edgecolor=custom_palette2[group_name],
                facecolor="white",
                alpha=0.9,
            ),  # Box style and colors
        )

    # Set limits for y-axis
    g.ax_joint.set_ylim(-0.3, 0.3)
    g.ax_joint.set_xlim(xlim_min, xlim_max)

    # Add a title to the plot
    g.fig.suptitle(f"{phenotype}: {region}", y=1.02)

    # Display the plot
    plt.show()

In [ ]:
clusters = [1, 2, 3 ,4]  # Example clusters
regions = [
    "Right Middle Temporal Gyrus",
    "Left Superior Temporal Gyrus",
    "Left Precentral Gyrus",
    "Left Angular Gyrus",
]  # Example regions
custom_palette = ["#A1AF58", "#73AFF1"]  # Custom palette with colors for each group
custom_palette2 = {"asd": "#A1AF58", "td": "#73AFF1"}  # Map Group names to colors

for phenotype in plotcorr_df["Phenotype"].unique():
    # Filter plotcorr_df for the current phenotype
    phenotype_df = plotcorr_df[plotcorr_df["Phenotype"] == phenotype].copy()
    # print(phenotype_df)

    for cluster, region in zip(clusters, regions):
        cluster_df = phenotype_df[
            phenotype_df["Cluster"] == cluster
        ].copy()  # Filter for current cluster

        #  Drop rows with NaN in 'Score' column
        cluster_df = cluster_df.dropna(subset=["Score"])
        print(cluster_df)

        plot_cluster_correlation(
            cluster_df, phenotype, cluster, region, custom_palette, custom_palette2
        )